# Glacier grids from SGI or GLAMOS:

Creates monthly grid files for the MBM to make PMB predictions over the whole glacier grid. The files come from the SGI grid and use OGGM topography. Computing takes a long time because of the conversion to monthly format.
## Setting up:

In [ ]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../../')) # Add root of repo to import MBM

import pandas as pd
import warnings
from tqdm.notebook import tqdm
import re
import massbalancemachine as mbm
import geopandas as gpd
import matplotlib.pyplot as plt
import geopandas as gpd
import geopandas as gpd
import rasterio 
import rioxarray
from concurrent.futures import ProcessPoolExecutor, as_completed
from pathlib import Path
import seaborn as sns

# scripts
from scripts.utils import *
from scripts.glamos import *
from scripts.geo_data import *
# from scripts.xgb_helpers import *
from scripts.config_CH import *
from scripts.geodetic import *

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
seed_all(cfg.seed)
free_up_cuda()  # in case no memory

# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)

# Climate columns
vois_climate = [
    't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]
# Topographical columns
voi_topographical = [
    "aspect",
    "slope",
    "hugonnet_dhdt",
    "consensus_ice_thickness",
    "millan_v",
    "topo",
]

In [ ]:
glaciers_glamos_dem = os.listdir(
    os.path.join(cfg.dataPath, path_GLAMOS_topo, 'lv95/'))

geodetic_mb = get_geodetic_MB(cfg)

# get years per glacier
years_start_per_gl = geodetic_mb.groupby(
    'glacier_name')['Astart'].unique().apply(list).to_dict()
years_end_per_gl = geodetic_mb.groupby('glacier_name')['Aend'].unique().apply(
    list).to_dict()

periods_per_glacier, geoMB_per_glacier = build_periods_per_glacier(geodetic_mb)
periods_per_glacier['silvretta']


## GLAMOS grids:

For the geodetic MB and gridded MB products computed by GLAMOS, they did not use the SGI grids (from 2015) but their own yearly DEMs. They're not available for all years, but we still compute monthly grids for these available glaciers and years, in order to make the comparison with geodetic MB fairer.

In [ ]:
# gdirs, rgidf = initialize_oggm_glacier_directories(
#     cfg,
#     rgi_region="11",
#     rgi_version="62",
#     base_url=
#     "https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/2025.6/elev_bands_w_data/",
#     log_level='WARNING',
#     task_list=None,
# )
# df_missing = export_oggm_grids(cfg, gdirs)

## Example of one glacier:

In [ ]:
glacier_name = 'gietro'
sgi_id, rgi_id, rgi_shp = get_rgi_sgi_ids(cfg, glacier_name)

folder_path = os.path.join(cfg.dataPath, path_GLAMOS_topo, 'lv95',
                           glacier_name)

# Example file
fileName = 'gl_2023_lv95.grid'
metadata, grid_data = load_grid_file(folder_path + '/' + fileName)

# Convert to xarray
dem_y = convert_to_xarray_geodata(grid_data, metadata)

# Transform the coordinates to WGS84
dem_wgs84_y = transform_xarray_coords_lv95_to_wgs84(dem_y)

# Create a mask where 'elevation' is not NaN (1 if not NaN, 0 if NaN)
ds_gl = xr.Dataset({'dem': dem_wgs84_y})
ds_gl["glacier_mask"] = ds_gl["dem"].notnull().astype(np.uint8)

dx = abs(ds_gl.x[1] - ds_gl.x[0]).values
dy = abs(ds_gl.y[1] - ds_gl.y[0]).values
print(f"Cell size of GLAMOS DEM: {dx} x {dy} meters")

# Extract SGI topo and aspect over GLAMOS DEM
ds = xr_GLAMOS_masked_topo(cfg, sgi_id, ds_gl)

# Coarson to 50 m resolution if needed
ds = coarsen_DS(ds)
dx_m, dy_m = get_res_from_degrees(ds)
print(f"Coarsened ds resolution: {dx_m} x {dy_m} meters")

# Plot the masked data
fig, axs = plt.subplots(1, 4, figsize=(15, 6))
ds.masked_aspect.plot(ax=axs[0], cmap='twilight_shifted', add_colorbar=False)
ds.masked_slope.plot(ax=axs[1], cmap='cividis', add_colorbar=False)
ds.masked_elev.plot(ax=axs[2], cmap='terrain', add_colorbar=False)
ds.glacier_mask.plot(ax=axs[3], cmap='binary', add_colorbar=False)

axs[0].set_title("Aspect")
axs[1].set_title("Slope")
axs[2].set_title("DEM")
axs[3].set_title("Glacier mask")
plt.tight_layout()

### Geotifs of DEMs:

In [ ]:
RUN = False
if RUN:
    glaciers_glamos_dems = os.listdir(
        os.path.join(cfg.dataPath, path_GLAMOS_topo, 'lv95'))

    path_out_tiff = os.path.join(cfg.dataPath,
                                 "GLAMOS/topo/GLAMOS_DEM/DEMs_geotiff_lv95/")
    os.makedirs(path_out_tiff, exist_ok=True)
    emptyfolder(path_out_tiff)

    for glacier_name in tqdm(glaciers_glamos_dems, desc="Processing glaciers"):

        sgi_id, rgi_id, rgi_shp = get_rgi_sgi_ids(cfg, glacier_name)

        folder_path = os.path.join(
            cfg.dataPath, path_GLAMOS_topo, 'lv95',
            'stanna' if glacier_name == 'sanktanna' else glacier_name)

        # Regular expression to extract years from filenames
        pattern = re.compile(r'gl_(\d{4})_lv95\.grid')

        # Extract available years from filenames
        years = sorted({
            int(match.group(1))
            for filename in os.listdir(folder_path)
            if (match := pattern.match(filename))
        })
        for i, year in enumerate(years):
            file_name = f'gl_{year}_lv95.grid'
            file_path = os.path.join(folder_path, file_name)

            # Load grid file
            metadata, grid_data = load_grid_file(file_path)

            # Convert to xarray
            masked_dem = convert_to_xarray_geodata(grid_data, metadata)

            # --- Attach CRS and write GeoTIFF ---
            masked_dem = masked_dem.rio.write_crs("EPSG:2056", inplace=True)

            # Prepare output folder
            out_tif = os.path.join(path_out_tiff, f"{glacier_name}_{year}.tif")
            masked_dem.rio.to_raster(
                out_tif,
                dtype="float32",
                compress="LZW",
                BIGTIFF="IF_SAFER",
                tiled=True,
                predictor=3,  # better compression for float rasters
            )

## Yearly masked grids - xarrays:
Save a .zarr xarray per glacier per year (not in monthly format) needed in the MBM later.

In [ ]:
# ---- pull any non-picklable cfg bits NOW into plain strings/ints ----
DATA_ROOT = cfg.dataPath  # assume this is just a string
PATH_GLAMOS_TOPO = path_GLAMOS_topo  # e.g. "GLAMOS/topo/..."
PATH_XR_SVF = os.path.join(DATA_ROOT, "GLAMOS/topo/GLAMOS_DEM",
                           "svf_nc_latlon")
PATH_XR_GRIDS = os.path.join(DATA_ROOT, PATH_GLAMOS_TOPO, "xr_masked_grids")

RUN = False
if RUN:
    # ensure clean output folder if you want a fresh run
    emptyfolder(PATH_XR_GRIDS)

    # ---- Build task list (glacier, year) ----
    glaciers_root = os.path.join(DATA_ROOT, PATH_GLAMOS_TOPO, 'lv95')
    glacier_names = os.listdir(glaciers_root)

    pattern = re.compile(r'gl_(\d{4})_lv95\.grid')

    tasks = []
    for glacier_name in glacier_names:
        folder_path = os.path.join(
            glaciers_root,
            'stanna' if glacier_name == 'sanktanna' else glacier_name)
        if not os.path.isdir(folder_path):
            continue
        years = sorted({
            int(m.group(1))
            for fn in os.listdir(folder_path) if (m := pattern.match(fn))
        })
        for year in years:
            if year >= 1951:
                tasks.append((glacier_name, year))

    print(f"Submitting {len(tasks)} tasks...")

    # ---- Run in parallel ----
    max_workers = max(1, (os.cpu_count() or 4) - 1)
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        futs = [
            ex.submit(process_yearly_gl_sgi_grid, cfg, g, y, DATA_ROOT,
                      PATH_GLAMOS_TOPO, PATH_XR_GRIDS, PATH_XR_SVF)
            for (g, y) in tasks
        ]

        for fut in as_completed(futs):
            res = fut.result()
            results.append(res)
            print(res)

    # (optional) summarize
    n_ok = sum(r.startswith("OK ") for r in results)
    n_err = sum(r.startswith("ERROR ") for r in results)
    n_skip = sum(r.startswith("SKIP ") for r in results)
    print(f"Done. OK={n_ok}, SKIP={n_skip}, ERROR={n_err}")

In [ ]:
# Plot the masked data
ds = xr.open_zarr(os.path.join(PATH_XR_GRIDS, 'aletsch_2016.zarr'))

fig, axs = plt.subplots(1, 5, figsize=(20, 6))
ds.masked_aspect.plot(ax=axs[0], cmap='twilight_shifted', add_colorbar=True)
ds.masked_slope.plot(ax=axs[1], cmap='cividis', add_colorbar=True)
ds.masked_elev.plot(ax=axs[2], cmap='terrain', add_colorbar=True)
ds.svf.plot(ax=axs[3], cmap='binary', add_colorbar=False)
ds.glacier_mask.plot(ax=axs[4], cmap='binary', add_colorbar=False)

axs[0].set_title("Aspect")
axs[1].set_title("Slope")
axs[2].set_title("DEM")
axs[3].set_title("Skyview factor")
axs[4].set_title("Glacier mask")
plt.tight_layout()
fig.show()

In [ ]:
# Plot the masked data
ds = xr.open_zarr(os.path.join(PATH_XR_GRIDS, 'gietro_2016.zarr'))

fig, axs = plt.subplots(2, 3, figsize=(15, 10))
ds.masked_aspect.plot(ax=axs[0, 0], cmap='twilight_shifted', add_colorbar=True)
ds.masked_slope.plot(ax=axs[0, 1], cmap='cividis', add_colorbar=True)
ds.masked_elev.plot(ax=axs[0, 2], cmap='terrain', add_colorbar=True)
ds.svf.plot(ax=axs[1, 0], cmap='binary', add_colorbar=False)
ds.glacier_mask.plot(ax=axs[1, 1], cmap='binary', add_colorbar=False)

axs[0, 0].set_title("Aspect")
axs[0, 1].set_title("Slope")
axs[0, 2].set_title("DEM")
axs[1, 0].set_title("Skyview factor")
axs[1, 1].set_title("Glacier mask")
plt.tight_layout()
fig.show()

## Monthly masked grids - dataframes:

In [ ]:
import scripts.geo_data.grid_inputs as grid_inputs

# ------------- pull only simple types out of cfg (picklable) -------------
DATA_ROOT = cfg.dataPath
PATH_XR_GRIDS = os.path.join(DATA_ROOT, path_GLAMOS_topo, 'xr_masked_grids')
OUT_FOLDER_ROOT = os.path.join(DATA_ROOT,
                               path_glacier_grid_glamos)  # parquet output root
ERA5_MONTHLY = os.path.join(DATA_ROOT, path_ERA5_raw,
                            'era5_monthly_averaged_data.nc')
ERA5_GEOPOT = os.path.join(DATA_ROOT, path_ERA5_raw,
                           'era5_geopotential_pressure.nc')
PCSR_ZARR = os.path.join(DATA_ROOT, path_pcsr, 'zarr/')

# If you have an RGI outlines shapefile path, use it here so workers load it once
RGI_OUTLINES_PATH = cfg.dataPath + path_rgi_outlines  # <-- set this to your outlines file path if available

VOIS_CLIMATE = [
    't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]
VOIS_TOPO = [
    "aspect", "slope", "hugonnet_dhdt", "consensus_ice_thickness", "millan_v",
    "topo", "svf"
]
TOO_SMALL = set(['vorab', 'blauschnee', 'joeri'])
ONLY_GEODETIC_YEARS = False

# meta columns used later (copy from cfg.metaData)
META_COLS = list(cfg.metaData)
OGGM_PATH = os.path.join(DATA_ROOT, path_OGGM)

In [ ]:
RUN = False
if RUN:
    # ============== build task list (glacier, year) ==============

    # ensure output root exists and (optionally) clean it
    os.makedirs(OUT_FOLDER_ROOT, exist_ok=True)
    emptyfolder(OUT_FOLDER_ROOT)  # if you truly want a fresh run

    # glaciers to consider
    all_glaciers = [g for g in years_start_per_gl.keys() if g not in TOO_SMALL]

    # map each glacier to missing years
    tasks = []
    pattern = re.compile(r'_(\d{4})\.zarr$')

    for glacier_name in all_glaciers:
        # zarr files present
        zarr_files = [
            f for f in os.listdir(PATH_XR_GRIDS)
            if f.startswith(f"{glacier_name}_") and f.endswith(".zarr")
        ]
        if not zarr_files:
            print(f"No GLAMOS DEM for {glacier_name}, skipping.")
            continue
        zarr_files.sort()

        # parquet folder
        out_folder = os.path.join(OUT_FOLDER_ROOT, glacier_name)
        os.makedirs(out_folder, exist_ok=True)

        # existing parquet years
        existing = {
            int(m.group(1))
            for f in os.listdir(out_folder)
            if (m := re.search(r'_grid_(\d{4})\.parquet$', f))
        }

        # geodetic period
        if glacier_name not in years_start_per_gl or glacier_name not in years_end_per_gl:
            print(f"Skipping {glacier_name}: missing start/end years")
            continue
        geodetic_start = years_start_per_gl[glacier_name][0]
        geodetic_end = years_end_per_gl[glacier_name][-1]

        # choose years
        for f in zarr_files:
            m = pattern.search(f)
            if not m:
                continue
            year = int(m.group(1))
            if year < 1951:
                continue
            if ONLY_GEODETIC_YEARS:
                if (year in range(geodetic_start, geodetic_end +
                                  1)) and (year not in existing):
                    tasks.append((glacier_name, year))
            else:
                if year not in existing:
                    tasks.append((glacier_name, year))

    print(f"Submitting {len(tasks)} tasks…")

    # ============== run in parallel ==============

    max_workers = max(1, (os.cpu_count() or 4) - 1)
    results = []
    n_ok = n_err = n_skip = 0
    start_month = "10"

    log_dir = Path("logs/errors")
    log_dir.mkdir(parents=True, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_path = log_dir / f"monthly_grids_{ts}.log"
    err_path = log_dir / f"monthly_grids_{ts}_ERRORS.log"

    with open(log_path, "w", encoding="utf-8") as f:
        f.write(
            f"Run started: {datetime.now().isoformat(timespec='seconds')}\n")
        f.write(f"max_workers={max_workers}\n")
        f.write(f"n_tasks={len(tasks)}\n")
        f.write("-" * 80 + "\n")

    with ProcessPoolExecutor(
            max_workers=max_workers,
            initializer=grid_inputs.init_worker,
            initargs=(RGI_OUTLINES_PATH, ),
    ) as ex:

        futs = [
            ex.submit(
                grid_inputs.process_monthly_grids_gl,
                cfg,
                glacier_name,
                year,
                data_root=DATA_ROOT,
                path_xr_grids=PATH_XR_GRIDS,
                out_folder_root=OUT_FOLDER_ROOT,
                vois_climate=VOIS_CLIMATE,
                vois_topo=VOIS_TOPO,
                meta_cols=META_COLS,
                era5_monthly_path=ERA5_MONTHLY,
                era5_geopot_path=ERA5_GEOPOT,
                pcsr_zarr_root=PCSR_ZARR,
                oggm_path=OGGM_PATH,
                start_month=start_month,
                small_glaciers=TOO_SMALL,
            ) for glacier_name, year in tasks
        ]

        with tqdm(total=len(futs), desc="Processing gl-years",
                  unit="task") as pbar:
            for fut in as_completed(futs):
                try:
                    res = fut.result()
                except Exception as e:
                    res = f"ERROR <future>: {type(e).__name__}: {e}"

                # ---- update counters ----
                if res.startswith("OK "):
                    n_ok += 1
                elif res.startswith("ERROR "):
                    n_err += 1
                elif res.startswith("SKIP "):
                    n_skip += 1

                # ---- write SKIP/ERROR lines to logs ----
                if res.startswith(("SKIP ", "ERROR ")):
                    line = f"[{datetime.now().isoformat(timespec='seconds')}] {res}\n"
                    with open(log_path, "a", encoding="utf-8") as f:
                        f.write(line)

                    if res.startswith("ERROR "):
                        with open(err_path, "a", encoding="utf-8") as f:
                            f.write(line)

                # update progress bar with live stats
                pbar.set_postfix(ok=n_ok, skip=n_skip, err=n_err)
                pbar.update(1)

    # ---- final summary ----
    with open(log_path, "a", encoding="utf-8") as f:
        f.write("-" * 80 + "\n")
        f.write(
            f"Run finished: {datetime.now().isoformat(timespec='seconds')}\n")
        f.write(f"OK={n_ok}, SKIP={n_skip}, ERROR={n_err}\n")

    print(f"Done. OK={n_ok}, SKIP={n_skip}, ERROR={n_err}")
    print(f"Log saved to: {log_path}")
    if n_err:
        print(f"Errors-only log: {err_path}")

In [ ]:
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv,
                          "CH_wgms_dataset_all.csv")
df_stakes = pd.read_csv(stake_file)

# Load GLAMOS masked grid
glacier_name = 'adler'
year = 2016

month = 'sep'  # Example month, adjust as needed

folder_path = os.path.join(cfg.dataPath, path_glacier_grid_glamos,
                           glacier_name)
# load the dataset
df = pd.read_parquet(
    os.path.join(folder_path, f"{glacier_name}_grid_{year}.parquet"))
df = df[df.MONTHS == month]

stake_locs = df_stakes[df_stakes.GLACIER == glacier_name]

# Variables of interest
voi = [
    "aspect_sgi",
    "slope_sgi",
]
fig, axs = plt.subplots(3, 4, figsize=(15, 10))
voi = [
    't2m', 'tp', 'ELEVATION_DIFFERENCE', 'hugonnet_dhdt',
    'consensus_ice_thickness', 'millan_v', 'aspect_sgi', 'slope_sgi', 'pcsr',
    'svf'
]
axs = axs.flatten()
for i, var in enumerate(voi):
    sns.scatterplot(df,
                    x='POINT_LON',
                    y='POINT_LAT',
                    hue=var,
                    s=5,
                    alpha=0.5,
                    palette='twilight_shifted',
                    ax=axs[i])
    axs[i].set_title(var)

    # scatter stake location
    sns.scatterplot(stake_locs,
                    x='POINT_LON',
                    y='POINT_LAT',
                    color='red',
                    s=10,
                    alpha=0.5,
                    ax=axs[i])

plt.tight_layout()

#### Add padding from Aug_:

In [ ]:
OUT_FOLDER_ROOT = '/scratch-3/vmarijn/MassBalanceMachine/../data/GLAMOS/topo/gridded_topo_inputs/GLAMOS_grid_Aug_/'
RUN = True
if RUN:
    # ============== build task list (glacier, year) ==============

    # ensure output root exists and (optionally) clean it
    os.makedirs(OUT_FOLDER_ROOT, exist_ok=True)
    #emptyfolder(OUT_FOLDER_ROOT)  # if you truly want a fresh run

    # glaciers to consider
    #all_glaciers = [g for g in years_start_per_gl.keys() if g not in TOO_SMALL]
    all_glaciers = ['adler']

    # map each glacier to missing years
    tasks = []
    pattern = re.compile(r'_(\d{4})\.zarr$')

    for glacier_name in all_glaciers:
        # zarr files present
        zarr_files = [
            f for f in os.listdir(PATH_XR_GRIDS)
            if f.startswith(f"{glacier_name}_") and f.endswith(".zarr")
        ]
        if not zarr_files:
            print(f"No GLAMOS DEM for {glacier_name}, skipping.")
            continue
        zarr_files.sort()

        # parquet folder
        out_folder = os.path.join(OUT_FOLDER_ROOT, glacier_name)
        os.makedirs(out_folder, exist_ok=True)

        # existing parquet years
        existing = {
            int(m.group(1))
            for f in os.listdir(out_folder)
            if (m := re.search(r'_grid_(\d{4})\.parquet$', f))
        }

        # geodetic period
        if glacier_name not in years_start_per_gl or glacier_name not in years_end_per_gl:
            print(f"Skipping {glacier_name}: missing start/end years")
            continue
        geodetic_start = years_start_per_gl[glacier_name][0]
        geodetic_end = years_end_per_gl[glacier_name][-1]

        # choose years
        for f in zarr_files:
            m = pattern.search(f)
            if not m:
                continue
            year = int(m.group(1))
            if year < 1951:
                continue
            if ONLY_GEODETIC_YEARS:
                if (year in range(geodetic_start, geodetic_end +
                                  1)) and (year not in existing):
                    tasks.append((glacier_name, year))
            else:
                if year not in existing:
                    tasks.append((glacier_name, year))

    print(f"Submitting {len(tasks)} tasks…")

    # ============== run in parallel ==============

    max_workers = max(1, (os.cpu_count() or 4) - 1)
    results = []
    n_ok = n_err = n_skip = 0
    start_month = "08"

    log_dir = Path("logs/errors")
    log_dir.mkdir(parents=True, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_path = log_dir / f"monthly_grids_aug_{ts}.log"
    err_path = log_dir / f"monthly_grids_aug_{ts}_ERRORS.log"

    with open(log_path, "w", encoding="utf-8") as f:
        f.write(
            f"Run started: {datetime.now().isoformat(timespec='seconds')}\n")
        f.write(f"max_workers={max_workers}\n")
        f.write(f"n_tasks={len(tasks)}\n")
        f.write("-" * 80 + "\n")

    with ProcessPoolExecutor(
            max_workers=max_workers,
            initializer=grid_inputs.init_worker,
            initargs=(RGI_OUTLINES_PATH, ),
    ) as ex:

        futs = [
            ex.submit(
                grid_inputs.process_monthly_grids_gl,
                cfg,
                glacier_name,
                year,
                data_root=DATA_ROOT,
                path_xr_grids=PATH_XR_GRIDS,
                out_folder_root=OUT_FOLDER_ROOT,
                vois_climate=VOIS_CLIMATE,
                vois_topo=VOIS_TOPO,
                meta_cols=META_COLS,
                era5_monthly_path=ERA5_MONTHLY,
                era5_geopot_path=ERA5_GEOPOT,
                pcsr_zarr_root=PCSR_ZARR,
                oggm_path=OGGM_PATH,
                start_month=start_month,
                small_glaciers=TOO_SMALL,
            ) for glacier_name, year in tasks
        ]

        with tqdm(total=len(futs), desc="Processing gl-years",
                  unit="task") as pbar:
            for fut in as_completed(futs):
                try:
                    res = fut.result()
                except Exception as e:
                    res = f"ERROR <future>: {type(e).__name__}: {e}"

                # ---- update counters ----
                if res.startswith("OK "):
                    n_ok += 1
                elif res.startswith("ERROR "):
                    n_err += 1
                elif res.startswith("SKIP "):
                    n_skip += 1

                # ---- write SKIP/ERROR lines to logs ----
                if res.startswith(("SKIP ", "ERROR ")):
                    line = f"[{datetime.now().isoformat(timespec='seconds')}] {res}\n"
                    with open(log_path, "a", encoding="utf-8") as f:
                        f.write(line)

                    if res.startswith("ERROR "):
                        with open(err_path, "a", encoding="utf-8") as f:
                            f.write(line)

                # update progress bar with live stats
                pbar.set_postfix(ok=n_ok, skip=n_skip, err=n_err)
                pbar.update(1)

    # ---- final summary ----
    with open(log_path, "a", encoding="utf-8") as f:
        f.write("-" * 80 + "\n")
        f.write(
            f"Run finished: {datetime.now().isoformat(timespec='seconds')}\n")
        f.write(f"OK={n_ok}, SKIP={n_skip}, ERROR={n_err}\n")

    print(f"Done. OK={n_ok}, SKIP={n_skip}, ERROR={n_err}")
    print(f"Log saved to: {log_path}")
    if n_err:
        print(f"Errors-only log: {err_path}")

##### Example:

In [ ]:
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv,
                          "CH_wgms_dataset_all.csv")
df_stakes = pd.read_csv(stake_file)

# Load GLAMOS masked grid
glacier_name = 'adler'
year = 2016

month = 'aug'  # Example month, adjust as needed

# folder_path = os.path.join(cfg.dataPath, path_glacier_grid_glamos,
#                            glacier_name)
# load the dataset
df = pd.read_parquet(
    os.path.join(OUT_FOLDER_ROOT, glacier_name, f"{glacier_name}_grid_{year}.parquet"))
df = df[df.MONTHS == month]

stake_locs = df_stakes[df_stakes.GLACIER == glacier_name]

# Variables of interest
voi = [
    "aspect_sgi",
    "slope_sgi",
]
fig, axs = plt.subplots(3, 4, figsize=(15, 10))
voi = [
    't2m', 'tp', 'ELEVATION_DIFFERENCE', 'hugonnet_dhdt',
    'consensus_ice_thickness', 'millan_v', 'aspect_sgi', 'slope_sgi', 'pcsr',
    'svf'
]
axs = axs.flatten()
for i, var in enumerate(voi):
    sns.scatterplot(df,
                    x='POINT_LON',
                    y='POINT_LAT',
                    hue=var,
                    s=5,
                    alpha=0.5,
                    palette='twilight_shifted',
                    ax=axs[i])
    axs[i].set_title(var)

    # scatter stake location
    sns.scatterplot(stake_locs,
                    x='POINT_LON',
                    y='POINT_LAT',
                    color='red',
                    s=10,
                    alpha=0.5,
                    ax=axs[i])

plt.tight_layout()